# A2A (Agent-to-Agent) Implementation Tutorial 🤖🔗🤖

Welcome to your first A2A implementation! This notebook will guide you step-by-step through building your own AI agents that can communicate with each other.

## What You'll Learn
1. ✅ Setting up your environment
2. ✅ Understanding A2A core concepts
3. ✅ Building a simple "Hello World" agent
4. ✅ Creating an Agent Card (the agent's identity)
5. ✅ Implementing message handling
6. ✅ Starting an A2A server
7. ✅ Creating a client to interact with your agent
8. ✅ Streaming responses
9. ✅ Multi-turn conversations
10. ✅ Advanced LLM integration with LangGraph

## Prerequisites
- Python 3.10 or higher installed
- Basic Python knowledge
- Terminal/Command Prompt access
- An internet connection

**Let's get started! 🚀**

## Section 1: Environment Setup

Before we write any code, we need to set up our Python environment. This section will guide you through installing all the required libraries.

### What We Need
- **a2a**: The A2A Python SDK (core library)
- **starlette**: Web framework for our agent server
- **uvicorn**: ASGI server to run our application
- **httpx**: HTTP client for making requests
- **pydantic**: Data validation (included with a2a)

### Important Notes
⚠️ **First-time setup**: If you haven't set up A2A before, you'll need to install it from the repository.  
⚠️ **Virtual environment**: It's recommended to use a virtual environment to avoid conflicts.

In [1]:
# Check Python version first
# A2A requires Python 3.10 or higher

import sys
print(f"Python version: {sys.version}")
print(f"Python version info: {sys.version_info}")

# Verify we have the right version
if sys.version_info >= (3, 10):
    print("✅ Python version is compatible!")
else:
    print("❌ ERROR: Please upgrade to Python 3.10 or higher")
    print("   Download from: https://www.python.org/downloads/")

Python version: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 16:37:03) [MSC v.1929 64 bit (AMD64)]
Python version info: sys.version_info(major=3, minor=13, micro=5, releaselevel='final', serial=0)
✅ Python version is compatible!


### Install Required Packages

**Note**: In this notebook, we'll install packages directly. In a real project, you'd typically:
1. Create a virtual environment: `python -m venv .venv`
2. Activate it: `.venv\Scripts\activate` (Windows) or `source .venv/bin/activate` (Mac/Linux)
3. Install from requirements.txt: `pip install -r requirements.txt`

For now, let's install the minimum packages we need:

In [2]:
# Install the core A2A packages
# This will take a minute or two to complete

# Uncomment the lines below to install (run once)
# !pip install --quiet starlette uvicorn httpx pydantic

# For the A2A SDK, you'll need to clone and install from the repository
# Since we're in a notebook, we'll assume it's already installed
# If not, follow the setup guide in the markdown file

# Verify the installations
try:
    import starlette
    import uvicorn
    import httpx
    import pydantic
    print("✅ All required packages are installed!")
    print(f"   - Starlette version: {starlette.__version__}")
    print(f"   - Uvicorn version: {uvicorn.__version__}")
    print(f"   - HTTPX version: {httpx.__version__}")
    print(f"   - Pydantic version: {pydantic.__version__}")
except ImportError as e:
    print(f"❌ Missing package: {e}")
    print("   Please install with: pip install starlette uvicorn httpx pydantic")

✅ All required packages are installed!
   - Starlette version: 0.48.0
   - Uvicorn version: 0.37.0
   - HTTPX version: 0.28.1
   - Pydantic version: 2.11.7


## Section 2: Understanding A2A Protocol Basics

Before we code, let's understand what we're building.

### What is A2A?
**A2A (Agent-to-Agent)** is a communication protocol that lets AI agents talk to each other, regardless of:
- What programming language they're written in
- What framework they use (LangChain, CrewAI, etc.)
- Who built them

Think of it like HTTP for websites - it's a standard way for agents to communicate.

### Key Concepts

#### 1. **Agent** 
An AI system that can:
- Receive messages
- Process them (maybe using an LLM)
- Send back responses

#### 2. **Agent Card**
Like a business card for your agent. It tells others:
- Who you are (name, description)
- Where to reach you (URL)
- What you can do (skills)
- How to authenticate

#### 3. **Message**
A single communication between agents:
```
User/Agent → "What's the weather?" → Your Agent
Your Agent → "It's sunny, 72°F" → User/Agent
```

#### 4. **Task**
A unit of work. For long-running operations:
- Has a unique ID
- Tracks state (submitted → working → completed)
- Can have artifacts (results)
- Supports multi-turn conversations

#### 5. **Part**
A piece of content in a message:
- Text: `"Hello world"`
- Files: PDFs, images, etc.
- Data: JSON objects

### Communication Flow

```
┌─────────┐         ┌─────────────┐         ┌──────────┐
│  Client │─────────│   A2A       │─────────│  Your    │
│         │  HTTP   │  Protocol   │  Code   │  Agent   │
│         │─────────│   Layer     │─────────│  Logic   │
└─────────┘         └─────────────┘         └──────────┘
   │                                              │
   │  1. Send Message                            │
   │─────────────────────────────────────────────▶│
   │                                              │
   │  2. Agent Processes (maybe calls LLM)       │
   │                                              │
   │  3. Send Response                            │
   │◀─────────────────────────────────────────────│
```

## Section 3: Import Required Libraries

Now let's import everything we need. I'll explain what each library does.

In [15]:
# Standard library imports
import asyncio
from typing import List, Optional, Dict, Any
import json

try:
    # Core A2A types
    from a2a.types import (
        AgentCapabilities,
        AgentCard,
        AgentSkill,
        Task,
        TaskState,
        TaskStatus,
        Message,
        Part,
        SendMessageRequest,
        MessageSendParams
    )
    
    # Server components - fixed import locations!
    from a2a.server.apps import A2AStarletteApplication
    from a2a.server.request_handlers import DefaultRequestHandler
    from a2a.server.agent_execution import RequestContext, AgentExecutor
    from a2a.server.events import EventQueue  # EventQueue is here!
    
    print("✅ All A2A imports successful!")
    print(f"   - Core types: AgentCapabilities, AgentCard, AgentSkill, Task, Message, Part")
    print(f"   - Server: A2AStarletteApplication, DefaultRequestHandler")
    print(f"   - Execution: RequestContext, AgentExecutor, EventQueue")
    
except ImportError as e:
    print(f"❌ A2A SDK not found: {e}")
    print("\n⚠️  Please run the installation cells above to install the A2A SDK")
    print("    Then restart the kernel (Kernel → Restart Kernel)")
    raise

✅ All A2A imports successful!
   - Core types: AgentCapabilities, AgentCard, AgentSkill, Task, Message, Part
   - Server: A2AStarletteApplication, DefaultRequestHandler
   - Execution: RequestContext, AgentExecutor, EventQueue


## Installing A2A SDK - Step by Step

Since the A2A SDK is not available on PyPI yet, we need to install it from the GitHub repository. Let's do this step by step!

In [4]:
# Step 1: Clone the A2A samples repository
# This will download all the example code and the A2A SDK

import os

# Check if the repository already exists
repo_path = os.path.join(os.getcwd(), 'a2a-samples')

if os.path.exists(repo_path):
    print("✅ Repository already exists at:", repo_path)
    print("   Skipping clone step.")
else:
    print("📥 Cloning A2A samples repository...")
    print("   This may take a minute...")
    
# We'll use the terminal to clone
# Run the cell below to execute the git clone command

📥 Cloning A2A samples repository...
   This may take a minute...


In [11]:
# Step 2: Clone the repository using git
# This downloads the A2A SDK and all examples

!git clone https://github.com/a2aproject/a2a-samples.git -b main --depth 1

fatal: destination path 'a2a-samples' already exists and is not an empty directory.


In [12]:
# Step 3: Install the A2A SDK and all dependencies
# This installs from the requirements.txt file in the repository

print("📦 Installing A2A SDK and dependencies...")
print("   This will install: a2a, starlette, uvicorn, httpx, langchain, and more")
print("   Please wait, this may take 1-2 minutes...\n")

!pip install -r a2a-samples/samples/python/requirements.txt

📦 Installing A2A SDK and dependencies...
   This will install: a2a, starlette, uvicorn, httpx, langchain, and more
   Please wait, this may take 1-2 minutes...



In [13]:
# Step 4: Verify the installation
# Let's check if everything is installed correctly

print("🔍 Verifying A2A SDK installation...\n")

try:
    import a2a
    print("✅ A2A SDK successfully installed!")
    
    # Check the version
    if hasattr(a2a, '__version__'):
        print(f"   Version: {a2a.__version__}")
    
    # Try importing key components
    from a2a.types import AgentCard, AgentSkill, Message
    from a2a.server.apps import A2AStarletteApplication
    from a2a.client import A2AClient
    
    print("✅ All core A2A components are available!")
    print("\n📦 Installed packages:")
    
    # Check other dependencies
    import starlette
    import uvicorn
    import httpx
    import pydantic
    
    print(f"   - A2A SDK: ✓")
    print(f"   - Starlette: {starlette.__version__}")
    print(f"   - Uvicorn: {uvicorn.__version__}")
    print(f"   - HTTPX: {httpx.__version__}")
    print(f"   - Pydantic: {pydantic.__version__}")
    
    print("\n🎉 Installation complete! You're ready to build A2A agents!")
    
except ImportError as e:
    print(f"❌ Installation verification failed: {e}")
    print("\n🔧 Troubleshooting:")
    print("   1. Make sure the git clone completed successfully")
    print("   2. Try running the pip install command again")
    print("   3. Restart the Jupyter kernel (Kernel → Restart Kernel)")
    print("   4. Re-run this cell")

🔍 Verifying A2A SDK installation...

✅ A2A SDK successfully installed!
✅ All core A2A components are available!

📦 Installed packages:
   - A2A SDK: ✓
   - Starlette: 0.48.0
   - Uvicorn: 0.37.0
   - HTTPX: 0.28.1
   - Pydantic: 2.11.7

🎉 Installation complete! You're ready to build A2A agents!


## Section 4: Build Your First Agent - "Hello World"

Let's start simple! We'll build an agent that just says "Hello World".

### Architecture Overview

Our agent will have 3 parts:
1. **Agent Logic** - The actual "brain" (returns "Hello World")
2. **Agent Executor** - Bridges A2A protocol with your logic
3. **Server Setup** - Exposes your agent over HTTP

Let's build each part!

### Part 1: The Agent Logic (Your Business Logic)

This is the simplest part - it's just your code that does something useful.  
For Hello World, it just returns a string.

In [10]:
class HelloWorldAgent:
    """
    This is your actual agent - the business logic.
    
    In a real application, this might:
    - Call an LLM (like GPT or Gemini)
    - Query a database
    - Call external APIs
    - Perform calculations
    
    For now, it just returns "Hello World"!
    """
    
    async def invoke(self) -> str:
        """
        The main method that does the work.
        
        Returns:
            str: The agent's response
        
        Why async? A2A is asynchronous because agents often do I/O:
        - Call APIs
        - Query databases
        - Call LLMs
        These can be slow, so async lets us handle many requests at once.
        """
        # In a real agent, you might do:
        # - result = await llm.generate("User's question")
        # - data = await database.query("SELECT ...")
        # - weather = await weather_api.get("Paris")
        
        # For now, just return a simple string
        return "Hello World"

# Create an instance of our agent
hello_agent = HelloWorldAgent()

# Test it locally (before making it A2A-compliant)
result = await hello_agent.invoke()
print(f"Agent says: {result}")

Agent says: Hello World


### Part 2: Define Agent Skills (What It Can Do)

Before we create the Agent Card, we need to define what our agent can do.  
This is called an **Agent Skill**.

In [11]:
# Create an Agent Skill
# Think of this as a menu item describing what the agent can do

hello_skill = AgentSkill(
    # Unique identifier for this skill
    # Should be lowercase with underscores
    id='hello_world',
    
    # Human-readable name
    # This is what users see
    name='Returns hello world',
    
    # Detailed description
    # Explain what this skill does and when to use it
    description='A simple skill that just returns the text "hello world". '
                'Use this to test if the agent is working.',
    
    # Tags for searchability
    # Other agents can search for skills by tags
    tags=['hello world', 'greeting', 'test'],
    
    # Example prompts that would trigger this skill
    # Helps other agents/users know how to interact
    examples=[
        'hi',
        'hello',
        'hello world',
        'say hello',
        'greet me'
    ]
)

print("✅ Agent Skill created!")
print(f"   Skill ID: {hello_skill.id}")
print(f"   Skill Name: {hello_skill.name}")
print(f"   Example prompts: {', '.join(hello_skill.examples)}")

✅ Agent Skill created!
   Skill ID: hello_world
   Skill Name: Returns hello world
   Example prompts: hi, hello, hello world, say hello, greet me


### Part 3: Create the Agent Card (Agent's Identity)

The **Agent Card** is like a digital business card. It tells others:
- Who you are
- Where to find you
- What you can do
- How to talk to you

This card is published at `/.well-known/agent-card.json` (standard location)

In [12]:
# Create the Agent Card
agent_card = AgentCard(
    # Agent's name (shown to users)
    name='Hello World Agent',
    
    # Brief description of what this agent does
    description='A simple demonstration agent that returns "Hello World". '
                'Perfect for testing A2A protocol implementation.',
    
    # URL where this agent is hosted
    # This must match where your server runs!
    # For local development: http://localhost:PORT
    url='http://localhost:9999/',
    
    # Version number (semantic versioning)
    version='1.0.0',
    
    # What input formats this agent accepts
    # Options: 'text', 'audio', 'image', 'video', 'file'
    # We only accept text for now
    default_input_modes=['text'],
    
    # What output formats this agent produces
    # Options: same as input
    # We only return text
    default_output_modes=['text'],
    
    # Capabilities - what protocol features this agent supports
    capabilities=AgentCapabilities(
        # Can we stream responses? Yes!
        # Streaming sends updates in real-time (like ChatGPT typing)
        streaming=True,
        
        # Can we send push notifications? No
        # Push notifications are webhooks for very long tasks
        pushNotifications=False,
        
        # Any custom protocol extensions? None
        extensions=[]
    ),
    
    # List of skills this agent has
    # An agent can have multiple skills
    skills=[hello_skill],
    
    # Does this agent have an extended card for authenticated users?
    # Extended cards can show premium features
    supports_authenticated_extended_card=False
)

print("✅ Agent Card created!")
print(f"\n📇 Agent Card Details:")
print(f"   Name: {agent_card.name}")
print(f"   URL: {agent_card.url}")
print(f"   Version: {agent_card.version}")
print(f"   Skills: {len(agent_card.skills)}")
print(f"   Input modes: {', '.join(agent_card.default_input_modes)}")
print(f"   Output modes: {', '.join(agent_card.default_output_modes)}")
print(f"   Supports streaming: {agent_card.capabilities.streaming}")

✅ Agent Card created!

📇 Agent Card Details:
   Name: Hello World Agent
   URL: http://localhost:9999/
   Version: 1.0.0
   Skills: 1
   Input modes: text
   Output modes: text
   Supports streaming: True


### Part 4: Create the Agent Executor (Bridge Between A2A and Your Code)

The **Agent Executor** is the bridge between:
- **A2A Protocol** (standardized messages, tasks, etc.)
- **Your Agent Logic** (the HelloWorldAgent we created)

It handles:
1. Receiving A2A protocol messages
2. Calling your agent
3. Converting results back to A2A format
4. Sending responses

In [16]:
class HelloWorldAgentExecutor(AgentExecutor):
    """
    The Agent Executor bridges A2A protocol with your agent logic.
    
    It implements two required methods:
    1. execute() - Handle incoming messages
    2. cancel() - Handle cancellation requests
    """
    
    def __init__(self):
        """
        Initialize the executor with your agent.
        
        In a real application, you might also:
        - Set up database connections
        - Initialize LLM clients
        - Load configuration
        """
        # Create an instance of our agent
        self.agent = HelloWorldAgent()
    
    async def execute(
        self,
        context: RequestContext,  # Info about the incoming request
        event_queue: EventQueue    # Where to send responses
    ) -> None:
        """
        Handle an incoming message and send a response.
        
        This is called when:
        - A client sends a message to your agent
        - A client continues a multi-turn conversation
        
        Args:
            context: Contains the incoming message, task ID, user info, etc.
            event_queue: Queue where you put responses to send back
        
        Flow:
            1. Get the user's message from context
            2. Call your agent logic
            3. Wrap result in A2A Message format
            4. Put it on the event queue
        """
        
        # Step 1: Call your agent (this does the actual work)
        result = await self.agent.invoke()
        
        # Step 2: Convert the result to an A2A Message
        # new_agent_text_message is a helper function that creates
        # a properly formatted Message with role='agent' and text content
        message = new_agent_text_message(result)
        
        # Step 3: Send the message back to the client
        # The event_queue is how you communicate results back
        # This is asynchronous - it doesn't block
        await event_queue.enqueue_event(message)
        
        # That's it! The A2A framework handles the rest:
        # - Serializing to JSON
        # - Sending over HTTP
        # - Error handling
    
    async def cancel(
        self,
        context: RequestContext,
        event_queue: EventQueue
    ) -> None:
        """
        Handle a cancellation request.
        
        This is called when a client wants to stop a running task.
        
        For example:
        - User clicks "Cancel" button
        - Task is taking too long
        - Client disconnects
        
        Our HelloWorld agent is instant, so we don't support cancellation.
        """
        # Raise an exception to indicate we don't support cancellation
        raise Exception('cancel not supported')
        
        # In a real agent with long-running tasks, you would:
        # - Stop any ongoing processing
        # - Clean up resources
        # - Send a Task with state=TaskState.cancelled

# Create an instance of our executor
hello_executor = HelloWorldAgentExecutor()

print("✅ Agent Executor created!")
print("   The executor is ready to handle A2A protocol messages.")

✅ Agent Executor created!
   The executor is ready to handle A2A protocol messages.


## Section 5: Start the A2A Server

Now we have all the pieces! Let's put them together and start the server.

### What Happens When We Start the Server:

1. **A2AStarletteApplication** creates a web application
2. It exposes the Agent Card at `/.well-known/agent-card.json`
3. It routes incoming requests to your RequestHandler
4. RequestHandler calls your AgentExecutor
5. Your executor calls your agent logic
6. Results flow back to the client

**Note**: Running a server in a Jupyter notebook is tricky because it blocks the cell.  
For this demo, we'll show the server code, but you'll run it in a separate Python file.

In [ ]:
# Server Setup Code
# This code shows how to create and configure the server
# In practice, you'll run this in a separate Python file (not in Jupyter)

# Step 1: Create the Request Handler
# This routes A2A protocol requests to your executor
request_handler = DefaultRequestHandler(
    # Your agent executor
    agent_executor=hello_executor,
    
    # Task store - where task state is saved
    # InMemoryTaskStore = stores in RAM (lost on restart)
    # For production, use a database-backed store
    task_store=InMemoryTaskStore(),
)

# Step 2: Create the A2A Server Application
# This is a Starlette web application with A2A support
server_app = A2AStarletteApplication(
    # The agent card to publish
    agent_card=agent_card,
    
    # The request handler to use
    http_handler=request_handler,
)

# Step 3: Build the Starlette app
# This creates the actual ASGI application
app = server_app.build()

print("✅ Server configured!")
print(f"\n🚀 To run the server, save this code to a file (e.g., server.py) and run:")
print(f"   uvicorn server:app --host 0.0.0.0 --port 9999")
print(f"\nOr use the shortcut:")
print(f"   python -c \"import uvicorn; uvicorn.run(app, host='0.0.0.0', port=9999)\"")
print(f"\n📍 Your agent will be available at: {agent_card.url}")
print(f"📇 Agent Card will be at: {agent_card.url}.well-known/agent-card.json")

### Create Server Script File

Let's create a complete server script that you can run outside this notebook:

In [ ]:
# Create a complete server script
server_code = """
# hello_world_server.py
# A complete A2A Hello World agent server

import uvicorn
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler, RequestContext, EventQueue
from a2a.server.tasks import InMemoryTaskStore
from a2a.server.agent_execution import AgentExecutor
from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from a2a.utils import new_agent_text_message


# 1. Agent Logic
class HelloWorldAgent:
    async def invoke(self) -> str:
        return 'Hello World'


# 2. Agent Executor
class HelloWorldAgentExecutor(AgentExecutor):
    def __init__(self):
        self.agent = HelloWorldAgent()

    async def execute(self, context: RequestContext, event_queue: EventQueue) -> None:
        result = await self.agent.invoke()
        message = new_agent_text_message(result)
        await event_queue.enqueue_event(message)

    async def cancel(self, context: RequestContext, event_queue: EventQueue) -> None:
        raise Exception('cancel not supported')


# 3. Agent Skill
skill = AgentSkill(
    id='hello_world',
    name='Returns hello world',
    description='just returns hello world',
    tags=['hello world'],
    examples=['hi', 'hello world'],
)

# 4. Agent Card
agent_card = AgentCard(
    name='Hello World Agent',
    description='Just a hello world agent',
    url='http://localhost:9999/',
    version='1.0.0',
    default_input_modes=['text'],
    default_output_modes=['text'],
    capabilities=AgentCapabilities(streaming=True),
    skills=[skill],
)

# 5. Create and Run Server
if __name__ == '__main__':
    request_handler = DefaultRequestHandler(
        agent_executor=HelloWorldAgentExecutor(),
        task_store=InMemoryTaskStore(),
    )

    server = A2AStarletteApplication(
        agent_card=agent_card,
        http_handler=request_handler,
    )

    print("🚀 Starting Hello World Agent...")
    print(f"📍 Agent URL: {agent_card.url}")
    print(f"📇 Agent Card: {agent_card.url}.well-known/agent-card.json")
    print("Press CTRL+C to stop")
    
    uvicorn.run(server.build(), host='0.0.0.0', port=9999)
"""

# Save the server script
with open('hello_world_server.py', 'w') as f:
    f.write(server_code)

print("✅ Server script created: hello_world_server.py")
print("\n📝 To run the server:")
print("   1. Open a terminal")
print("   2. Run: python hello_world_server.py")
print("   3. Keep it running (don't close the terminal)")
print("\n   You should see: 'Uvicorn running on http://0.0.0.0:9999'")
print("\n⚠️  The server will block until you press CTRL+C")

## Section 6: Create a Client to Test Your Agent

Now that we know how to build an agent, let's build a client to talk to it!

### What the Client Does:
1. Fetches the Agent Card (to learn about the agent)
2. Creates an A2A Client
3. Sends a message
4. Receives the response

**Assumption**: The server from the previous section is running on `http://localhost:9999`

### Step 1: Fetch the Agent Card

First, let's verify the server is running by fetching its Agent Card:

In [ ]:
# Test fetching the Agent Card
# This verifies the server is running and reachable

async def fetch_agent_card_simple(base_url: str):
    """
    Fetch and parse the Agent Card from a running agent.
    
    Args:
        base_url: The base URL of the agent (e.g., 'http://localhost:9999')
    
    Returns:
        dict: The agent card as a dictionary
    """
    # The Agent Card is always at this standard location
    card_url = f"{base_url}/.well-known/agent-card.json"
    
    # Use httpx to make an async HTTP request
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(card_url)
            response.raise_for_status()  # Raise error if status is 4xx or 5xx
            return response.json()
        except httpx.ConnectError:
            print(f"❌ Could not connect to {card_url}")
            print("   Make sure the server is running!")
            return None
        except Exception as e:
            print(f"❌ Error fetching agent card: {e}")
            return None

# Try to fetch the card
base_url = 'http://localhost:9999'
agent_card_data = await fetch_agent_card_simple(base_url)

if agent_card_data:
    print("✅ Successfully fetched Agent Card!")
    print(f"\n📇 Agent Card:")
    print(json.dumps(agent_card_data, indent=2))
else:
    print("\n⚠️  Could not fetch Agent Card.")
    print("   The server might not be running yet.")
    print("   Start the server first: python hello_world_server.py")

### Step 2: Send a Message Using the A2A Client

Now let's use the official A2A Client to send a message:

In [ ]:
async def send_message_to_agent(base_url: str, message_text: str):
    """
    Send a message to an A2A agent and get the response.
    
    Args:
        base_url: The agent's base URL
        message_text: The text to send
    
    Returns:
        The agent's response
    """
    async with httpx.AsyncClient() as httpx_client:
        # Step 1: Resolve the Agent Card
        # This fetches and parses the agent card
        print(f"📡 Connecting to agent at {base_url}...")
        resolver = A2ACardResolver(
            httpx_client=httpx_client,
            base_url=base_url,
        )
        
        try:
            agent_card = await resolver.resolve_agent_card()
            print(f"✅ Connected to: {agent_card.name}")
            print(f"   Description: {agent_card.description}")
            print(f"   Skills: {', '.join([s.name for s in agent_card.skills])}")
        except Exception as e:
            print(f"❌ Failed to resolve agent card: {e}")
            return None
        
        # Step 2: Create the A2A Client
        # This client knows how to send A2A protocol messages
        client = A2AClient(
            httpx_client=httpx_client,
            agent_card=agent_card
        )
        
        # Step 3: Create the message
        # This follows the A2A Message format
        print(f"\n📤 Sending message: \"{message_text}\"")
        message_data = {
            'message': {
                'role': 'user',  # We're the user
                'parts': [
                    {
                        'kind': 'text',  # Text content
                        'text': message_text
                    }
                ],
                'messageId': uuid4().hex,  # Unique ID for this message
            },
        }
        
        # Step 4: Create the request object
        request = SendMessageRequest(
            id=str(uuid4()),  # Request ID (for tracking)
            params=MessageSendParams(**message_data)
        )
        
        # Step 5: Send the message and get response
        try:
            response = await client.send_message(request)
            
            # The response is either a Message or a Task
            print(f"\n📥 Received response:")
            print(f"   Type: {response.__class__.__name__}")
            
            # If it's a message, extract the text
            if hasattr(response, 'parts'):
                for part in response.parts:
                    if hasattr(part, 'text'):
                        print(f"   Agent says: \"{part.text}\"")
            
            return response
        
        except Exception as e:
            print(f"❌ Error sending message: {e}")
            return None

# Test it!
response = await send_message_to_agent('http://localhost:9999', 'Hello!')

if response:
    print("\n✅ Message sent and response received successfully!")

## Section 7: Understanding Messages and Tasks

Let's dive deeper into the two types of responses you can get from an agent.

### Response Type 1: Message (Simple, Immediate Response)

A **Message** is for quick, instant responses:
- Agent responds immediately
- No tracking needed
- Like a simple function call

**Example**: "Hello World" → "Hello World"

### Response Type 2: Task (Long-Running Operation)

A **Task** is for operations that take time:
- Has a unique ID
- Tracks state (working → completed)
- Can be queried later
- Supports cancellation
- Can have multiple updates

**Example**: "Analyze this 100-page document" → Task (takes minutes)

In [ ]:
# Let's explore the anatomy of an A2A Message

# Creating a Message manually (without helpers)
from a2a.types import Message, Part, TextPart

manual_message = Message(
    # Unique identifier for this message
    message_id=str(uuid4()),
    
    # Who sent it? 'user' or 'agent'
    role='agent',
    
    # The content (can have multiple parts)
    parts=[
        Part(text="This is a text part"),
        # Could also have:
        # Part(file=FilePart(...))  - for files
        # Part(data=DataPart(...))  - for structured data
    ],
    
    # Optional: Link to a context (for multi-turn)
    context_id=None,
    
    # Optional: Link to a task
    task_id=None,
    
    # Optional: Custom metadata
    metadata={},
    
    # Optional: Protocol extensions
    extensions=[]
)

print("📧 Message Structure:")
print(f"   Message ID: {manual_message.message_id}")
print(f"   Role: {manual_message.role}")
print(f"   Number of parts: {len(manual_message.parts)}")
print(f"   First part type: {type(manual_message.parts[0])}")

# Now let's look at a Task
from a2a.types import Task, TaskStatus, TaskState, Artifact

manual_task = Task(
    # Unique task identifier
    id=str(uuid4()),
    
    # Context ID (groups related tasks)
    context_id=str(uuid4()),
    
    # Current status
    status=TaskStatus(
        # State enum: submitted, working, completed, failed, cancelled, etc.
        state=TaskState.working,
        
        # Optional status message
        message=Message(
            message_id=str(uuid4()),
            role='agent',
            parts=[Part(text="Processing your request...")]
        )
    ),
    
    # Results/outputs (filled when complete)
    artifacts=[],
    
    # Conversation history
    history=[],
    
    # Custom metadata
    metadata={}
)

print("\n📋 Task Structure:")
print(f"   Task ID: {manual_task.id}")
print(f"   Context ID: {manual_task.context_id}")
print(f"   State: {manual_task.status.state}")
print(f"   Status message: {manual_task.status.message.parts[0].text}")

## Section 8: Streaming Responses

Streaming is like watching ChatGPT type in real-time. Instead of waiting for the complete response, you get updates as they happen.

### Why Streaming?
- **Better UX**: Users see progress immediately
- **Long operations**: Don't time out waiting for complete response
- **Real-time feedback**: Show what the agent is doing

### How It Works
Instead of returning one response, the agent sends multiple events:
1. Task created
2. Status update: "Thinking..."
3. Status update: "Calling API..."
4. Artifact update: Partial result
5. Artifact update: More results
6. Final status: Completed

In [ ]:
# Let's create a streaming client

async def send_streaming_message(base_url: str, message_text: str):
    """
    Send a message and stream the response.
    
    This demonstrates how to receive real-time updates from the agent.
    """
    async with httpx.AsyncClient() as httpx_client:
        # Resolve agent card
        resolver = A2ACardResolver(
            httpx_client=httpx_client,
            base_url=base_url,
        )
        agent_card = await resolver.resolve_agent_card()
        
        # Create client
        client = A2AClient(
            httpx_client=httpx_client,
            agent_card=agent_card
        )
        
        # Create message
        message_data = {
            'message': {
                'role': 'user',
                'parts': [{'kind': 'text', 'text': message_text}],
                'messageId': uuid4().hex,
            },
        }
        
        # Create STREAMING request (different from regular send)
        from a2a.types import SendStreamingMessageRequest
        request = SendStreamingMessageRequest(
            id=str(uuid4()),
            params=MessageSendParams(**message_data)
        )
        
        # Send and stream responses
        print(f"📤 Sending (streaming): \"{message_text}\"")
        print("📥 Receiving stream:")
        print("-" * 50)
        
        # This returns an async generator - each iteration is an event
        stream_response = client.send_message_streaming(request)
        
        event_count = 0
        async for event in stream_response:
            event_count += 1
            print(f"\n[Event {event_count}]")
            
            # Each event could be:
            # - Task (initial task creation)
            # - TaskStatusUpdateEvent (status changed)
            # - TaskArtifactUpdateEvent (new result)
            
            print(f"  Type: {event.__class__.__name__}")
            
            # Extract and display relevant info
            if hasattr(event, 'status'):
                print(f"  Status: {event.status.state}")
                if event.status.message:
                    for part in event.status.message.parts:
                        if hasattr(part, 'text'):
                            print(f"  Message: {part.text}")
            
            if hasattr(event, 'artifact'):
                print(f"  Artifact ID: {event.artifact.artifact_id}")
                for part in event.artifact.parts:
                    if hasattr(part, 'text'):
                        print(f"  Content: {part.text}")
            
            # Check if this is the final event
            if hasattr(event, 'final') and event.final:
                print("  [FINAL EVENT]")
        
        print("-" * 50)
        print(f"✅ Stream complete! Received {event_count} events.")

# Test streaming (only if server is running)
# await send_streaming_message('http://localhost:9999', 'Hello!')

print("💡 Streaming example ready!")
print("   Uncomment the line above to test (server must be running)")

## Section 9: Multi-Turn Conversations

Multi-turn conversations let your agent have back-and-forth exchanges, just like chatting with ChatGPT.

### How It Works:

**Turn 1:**
```
User: "Convert 100 USD"
Agent: Task(state='input-required', message="To which currency?")
```

**Turn 2:**
```
User: "to EUR" (includes task_id from Turn 1)
Agent: Task(state='completed', artifacts=["100 USD = 85 EUR"])
```

The key is the **context_id** and **task_id** - they link the conversations together.

In [ ]:
# Example: Multi-turn conversation flow

print("🗣️  Multi-Turn Conversation Example")
print("=" * 60)

# Turn 1: Initial request (incomplete information)
print("\n👤 User: 'Convert 100 USD'")
print("   (No target currency specified)")
print("\n🤖 Agent Response:")
print("   {")
print("     'task': {")
print("       'id': 'task-123',")
print("       'context_id': 'ctx-abc',")
print("       'status': {")
print("         'state': 'input-required',")
print("         'message': 'To which currency would you like to convert?'")
print("       }")
print("     }")
print("   }")
print("\n   💡 Agent needs more information!")

# Turn 2: Follow-up with more info
print("\n" + "-" * 60)
print("\n👤 User: 'to EUR'")
print("   task_id: 'task-123'  ← Links to previous task")
print("   context_id: 'ctx-abc'  ← Same conversation")
print("\n🤖 Agent Response:")
print("   {")
print("     'task': {")
print("       'id': 'task-123',  ← Same task")
print("       'context_id': 'ctx-abc',  ← Same context")
print("       'status': {'state': 'completed'},")
print("       'artifacts': [")
print("         {")
print("           'name': 'conversion-result',")
print("           'parts': [{'text': '100 USD = 85 EUR'}]")
print("         }")
print("       ]")
print("     }")
print("   }")
print("\n   ✅ Task completed with full information!")

print("\n" + "=" * 60)
print("\n🔑 Key Concepts:")
print("   • context_id: Groups related messages/tasks")
print("   • task_id: Continues a specific task")
print("   • state='input-required': Agent needs more info")
print("   • Agent remembers previous messages via context")

## Section 10: Summary and Next Steps

Congratulations! 🎉 You've learned the fundamentals of A2A implementation!

### What You Built:
✅ A Hello World agent with Agent Logic  
✅ An Agent Card describing your agent  
✅ An Agent Executor bridging A2A protocol with your code  
✅ An A2A server exposing your agent over HTTP  
✅ A client to send messages to agents  
✅ Understanding of streaming and multi-turn conversations  

### Key Concepts Mastered:
- **Agent Card**: Identity and capabilities
- **Agent Skill**: Individual feature description
- **Message**: Single communication turn
- **Task**: Long-running operation with state
- **Part**: Content piece (text, file, data)
- **AgentExecutor**: Protocol bridge
- **Streaming**: Real-time updates
- **Multi-turn**: Conversational interactions

### Next Steps - Level Up Your Skills!

#### 1. **Try the LangGraph Example** (Advanced)
The tutorial files include a Currency Conversion agent that uses:
- Google Gemini LLM
- LangGraph for agent orchestration
- Real-time streaming
- Multi-turn conversations

**To try it:**
1. Get a Gemini API key from [Google AI Studio](https://aistudio.google.com/)
2. Set it in `.env`: `GOOGLE_API_KEY=your_key_here`
3. Navigate to `samples/python/agents/langgraph/app`
4. Run: `python __main__.py`

#### 2. **Build Your Own Agent Ideas**

**Easy Projects:**
- **Calculator Agent**: Evaluates math expressions
- **Time Agent**: Returns current time in different timezones
- **Random Quote Agent**: Returns inspiring quotes

**Intermediate Projects:**
- **Weather Agent**: Calls a weather API
- **News Agent**: Fetches latest news headlines
- **Translation Agent**: Translates text between languages

**Advanced Projects:**
- **RAG Agent**: Retrieves info from documents (LangChain + Vector DB)
- **Multi-Agent System**: Orchestrator + specialist agents
- **Code Assistant**: Generates and explains code

#### 3. **Explore Other Examples**
The `a2a-samples` repository has examples in:
- Python
- Java
- JavaScript
- C#

#### 4. **Read the Specification**
For production systems, read:
- Protocol Specification (specification folder)
- Security best practices
- Error handling patterns
- Authentication methods

#### 5. **Build Your Mini Project**
Now that you know:
- LangChain
- LangGraph
- MCP (Model Context Protocol)
- A2A (Agent-to-Agent)
- JSON

You're ready to build a complete multi-agent application! 🚀

**Project Ideas:**
- Travel planner (flight agent + hotel agent + activity agent)
- Research assistant (search agent + summarizer agent + writer agent)
- Customer service (classifier agent + FAQ agent + escalation agent)

### Quick Reference

#### Starting a Server
```python
import uvicorn
from a2a.server.apps import A2AStarletteApplication

server = A2AStarletteApplication(
    agent_card=your_agent_card,
    http_handler=your_request_handler,
)

uvicorn.run(server.build(), host='0.0.0.0', port=9999)
```

#### Creating a Client
```python
from a2a.client import A2AClient, A2ACardResolver

async with httpx.AsyncClient() as client:
    resolver = A2ACardResolver(
        httpx_client=client,
        base_url='http://localhost:9999'
    )
    agent_card = await resolver.resolve_agent_card()
    
    a2a_client = A2AClient(
        httpx_client=client,
        agent_card=agent_card
    )
```

#### Sending a Message
```python
from a2a.types import SendMessageRequest, MessageSendParams
from uuid import uuid4

request = SendMessageRequest(
    id=str(uuid4()),
    params=MessageSendParams(
        message={
            'role': 'user',
            'parts': [{'kind': 'text', 'text': 'Hello!'}],
            'messageId': uuid4().hex,
        }
    )
)

response = await a2a_client.send_message(request)
```

#### AgentExecutor Template
```python
from a2a.server.agent_execution import AgentExecutor

class MyAgentExecutor(AgentExecutor):
    async def execute(self, context, event_queue):
        # 1. Get user message
        user_msg = context.message
        
        # 2. Process (your logic here)
        result = await my_agent.process(user_msg)
        
        # 3. Send response
        message = new_agent_text_message(result)
        await event_queue.enqueue_event(message)
    
    async def cancel(self, context, event_queue):
        # Handle cancellation
        pass
```

In [ ]:
# Final Code: Complete Hello World Agent (All in One Cell)
# Copy this to a file called 'my_first_agent.py' and run it!

complete_agent_code = '''
"""
My First A2A Agent - Complete Implementation
Save as: my_first_agent.py
Run with: python my_first_agent.py
"""

import uvicorn
from uuid import uuid4
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities, 
    AgentCard, 
    AgentSkill
)
from a2a.utils import new_agent_text_message


# 1. Agent Logic (Your Business Logic)
class HelloWorldAgent:
    """The actual agent - replace this with your logic"""
    
    async def invoke(self) -> str:
        return 'Hello World from my first A2A agent! 🎉'


# 2. Agent Executor (Protocol Bridge)
class HelloWorldAgentExecutor(AgentExecutor):
    """Bridges A2A protocol with your agent"""
    
    def __init__(self):
        self.agent = HelloWorldAgent()
    
    async def execute(
        self,
        context: RequestContext,
        event_queue: EventQueue,
    ) -> None:
        # Call your agent
        result = await self.agent.invoke()
        
        # Convert to A2A format
        message = new_agent_text_message(result)
        
        # Send response
        await event_queue.enqueue_event(message)
    
    async def cancel(
        self,
        context: RequestContext,
        event_queue: EventQueue
    ) -> None:
        raise Exception('cancel not supported')


# 3. Define Agent Skill
skill = AgentSkill(
    id='hello_world',
    name='Returns hello world',
    description='A friendly greeting agent',
    tags=['hello', 'greeting', 'test'],
    examples=['hi', 'hello', 'hello world'],
)

# 4. Create Agent Card
agent_card = AgentCard(
    name='My First A2A Agent',
    description='A simple Hello World agent built with A2A protocol',
    url='http://localhost:9999/',
    version='1.0.0',
    default_input_modes=['text'],
    default_output_modes=['text'],
    capabilities=AgentCapabilities(streaming=True),
    skills=[skill],
)

# 5. Start Server
if __name__ == '__main__':
    print("=" * 60)
    print("🚀 Starting My First A2A Agent...")
    print("=" * 60)
    print(f"📇 Agent Name: {agent_card.name}")
    print(f"📍 URL: {agent_card.url}")
    print(f"📄 Agent Card: {agent_card.url}.well-known/agent-card.json")
    print(f"🔧 Skills: {', '.join([s.name for s in agent_card.skills])}")
    print("=" * 60)
    print("✅ Server is running!")
    print("💡 Test it: Run the client code from the notebook")
    print("⏹️  Stop: Press CTRL+C")
    print("=" * 60)
    
    # Create request handler
    request_handler = DefaultRequestHandler(
        agent_executor=HelloWorldAgentExecutor(),
        task_store=InMemoryTaskStore(),
    )
    
    # Create and run server
    server = A2AStarletteApplication(
        agent_card=agent_card,
        http_handler=request_handler,
    )
    
    uvicorn.run(server.build(), host='0.0.0.0', port=9999)
'''

# Save it
with open('my_first_agent.py', 'w', encoding='utf-8') as f:
    f.write(complete_agent_code)

print("✅ Complete agent code saved to: my_first_agent.py")
print("\n🎯 To run your agent:")
print("   1. Open a new terminal")
print("   2. Run: python my_first_agent.py")
print("   3. Test with the client code from this notebook")
print("\n🎉 You've completed the A2A tutorial!")
print("   You're ready to build amazing multi-agent systems!")